In [ ]:
import geopandas as gpd

# Load GeoJSON file
geojson_file = 'train.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'train.xlsx'
gdf.to_excel(xlsx, index=False)

# Load GeoJSON file
geojson_file = 'test.geojson'
gdf = gpd.read_file(geojson_file)


# Save DataFrame to XLSX
xlsx = 'test.xlsx'
gdf.to_excel(xlsx, index=False)

In [ ]:
import pandas as pd
#PREPROCESSING TEST PART
# Charger le fichier Excel
chemin_fichier_excel = 'test.xlsx'
df = pd.read_excel(chemin_fichier_excel)
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
colonnes_a_traiter = df.columns[2:32] 

# Calculer la médiane pour chaque colonne spécifiée
medianes = df[colonnes_a_traiter].median()

# Remplacer les cellules vides par les médianes correspondantes
df[colonnes_a_traiter] = df[colonnes_a_traiter].fillna(medianes)
def processv2(df):
    data = df['geometry']
    liste=[]
    for ligne in data:
        sl=[]
        ligne=ligne[10:-2]
        polygone = ligne.split(',')
        for element in polygone:
            a,b= [i for i in element.split(' ') if i != '']
            sl.append((float(a),float(b)))
        liste.append(sl)
    return liste

from shapely.geometry import Polygon
def calculer_aire_polygone(sommets):
    """
    Calcule l'aire d'un polygone à l'aide de la bibliothèque shapely.
    
    :param sommets: Liste des sommets du polygone sous forme de tuples (x, y)
    :return: L'aire du polygone
    """

    polygone = Polygon(sommets)
    aire = polygone.area
    return aire
def perim(sommets):
    polygone = Polygon(sommets)
# Calculer le périmètre du polygone
    perimetre = polygone.length
    return perimetre
def calculate_height_width_ratio(sommets):
    polygon = Polygon(sommets)
    bounds = polygon.bounds
    height = bounds[3] - bounds[1]  # ymax - ymin
    width = bounds[2] - bounds[0]   # xmax - xmin
    return height / width

def calculate_centroid_x(sommets):
    polygon = Polygon(sommets)
    centroid = polygon.centroid
    return centroid.x

def calculate_centroid_y(sommets):
    polygon = Polygon(sommets)
    centroid = polygon.centroid
    return centroid.y
# Calculer l'aire de chaque polygone dans la liste
liste_aires = [calculer_aire_polygone(polygone) for polygone in processv2(df)]
liste_perim = [perim(polygone) for polygone in processv2(df)]
liste_ratio = [calculate_height_width_ratio(polygone) for polygone in processv2(df)]
liste_centroid_x = [calculate_centroid_x(polygone) for polygone in processv2(df)]
liste_centroid_y = [calculate_centroid_y(polygone) for polygone in processv2(df)]
# Ajouter la nouvelle colonne au DataFrame
df['Area'] = liste_aires
df['Perimeter']=liste_perim
df['ratio']=liste_ratio
df['x_centroid']=liste_centroid_x
df['y_centroid']=liste_centroid_y
df = df.drop(['geometry'],axis=1)
df.iloc[:, 0] = df.iloc[:, 0].str.replace('N,A', 'NSP_urban')

df.iloc[:, 1] = df.iloc[:, 1].str.replace('N,A', 'NSP_geo')
one_hot_encoded = df['urban_type'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded)

one_hot_encoded = df['geography_type'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded)


colonnes_a_supprimer = ['urban_type', 'geography_type','index']  # Remplacez par les noms des colonnes à supprimer
df = df.drop(columns=colonnes_a_supprimer)

df['date0'] = pd.to_datetime(df['date0'], format='%d-%m-%Y')
df['date1'] = pd.to_datetime(df['date1'], format='%d-%m-%Y')
df['date2'] = pd.to_datetime(df['date2'], format='%d-%m-%Y')
df['date3'] = pd.to_datetime(df['date3'], format='%d-%m-%Y')
df['date4'] = pd.to_datetime(df['date4'], format='%d-%m-%Y')

# Calculer l'écart en nombre de jours entre les deux dates
df['Ecart_jours1'] = (df['date1'] - df['date0']).dt.days
df['Ecart_jours2'] = (df['date2'] - df['date1']).dt.days
df['Ecart_jours3'] = (df['date3'] - df['date2']).dt.days
df['Ecart_jours4'] = (df['date4'] - df['date3']).dt.days

colonnes_a_supprimer = ['date0','date1','date2','date3','date4']  
df = df.drop(columns=colonnes_a_supprimer)

# Appliquer le One-Hot Encoding sur la colonne 'Categories'
one_hot_encoded0 = df['change_status_date0'].str.get_dummies(sep=',')
one_hot_encoded1 = df['change_status_date1'].str.get_dummies(sep=',')
one_hot_encoded2 = df['change_status_date2'].str.get_dummies(sep=',')
one_hot_encoded3 = df['change_status_date3'].str.get_dummies(sep=',')
one_hot_encoded4 = df['change_status_date4'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded0,rsuffix='0')
df = df.join(one_hot_encoded1,rsuffix='1')
df = df.join(one_hot_encoded2,rsuffix='2')
df = df.join(one_hot_encoded3,rsuffix='3')
df = df.join(one_hot_encoded4,rsuffix='4')

colonnes_a_traiter=['Ecart_jours1','Ecart_jours2','Ecart_jours3','Ecart_jours4']
medianes = df[colonnes_a_traiter].median()

# Remplacer les cellules vides par les médianes correspondantes
df[colonnes_a_traiter] = df[colonnes_a_traiter].fillna(medianes)
colonnes_a_supprimer = ['change_status_date0','change_status_date1','change_status_date2','change_status_date3','change_status_date4'] 
df = df.drop(columns=colonnes_a_supprimer)
couleurs = ['red', 'blue', 'green']

# Créer une liste des dates à considérer (date1, date2, date3, date4, date5)
dates = ['date1', 'date2', 'date3', 'date4', 'date5']

# Parcourir chaque couleur
for couleur in couleurs:
    # Créer une liste pour stocker les moyennes de chaque ligne
    moyennes = []
    # Parcourir chaque ligne
    for index, row in df.iterrows():
        # Créer une liste pour stocker les valeurs de chaque date pour cette couleur
        valeurs_couleur = []
        # Parcourir chaque date
        for date in dates:
            # Récupérer le nom de la colonne
            nom_colonne = f'img_{couleur}_mean_{date}'
            # Ajouter la valeur de la colonne à la liste des valeurs de cette couleur
            valeurs_couleur.append(row[nom_colonne])
        # Calculer la moyenne des valeurs pour cette ligne
        moyenne_ligne = sum(valeurs_couleur) / len(valeurs_couleur)
        # Ajouter la moyenne à la liste des moyennes
        moyennes.append(moyenne_ligne)
    # Créer une nouvelle colonne dans le DataFrame pour stocker les moyennes de cette couleur
    nom_nouvelle_colonne = f'moyenne_{couleur}'
    df[nom_nouvelle_colonne] = moyennes
for couleur in couleurs:
    # Créer une liste pour stocker les moyennes de chaque ligne
    moyennes = []
    # Parcourir chaque ligne
    for index, row in df.iterrows():
        # Créer une liste pour stocker les valeurs de chaque date pour cette couleur
        valeurs_couleur = []
        # Parcourir chaque date
        for date in dates:
            # Récupérer le nom de la colonne
            nom_colonne = f'img_{couleur}_std_{date}'
            # Ajouter la valeur de la colonne à la liste des valeurs de cette couleur
            valeurs_couleur.append(row[nom_colonne])
        # Calculer la moyenne des valeurs pour cette ligne
        moyenne_ligne = sum(valeurs_couleur) / len(valeurs_couleur)
        # Ajouter la moyenne à la liste des moyennes
        moyennes.append(moyenne_ligne)
    # Créer une nouvelle colonne dans le DataFrame pour stocker les moyennes de cette couleur
    nom_nouvelle_colonne = f'std_{couleur}'
    df[nom_nouvelle_colonne] = moyennes
df.to_excel('test_final5.xlsx',index=False)

In [ ]:
import pandas as pd
#PREPROCESSING TRAIN PART
# Charger le fichier Excel
chemin_fichier_excel = 'train.xlsx'
df = pd.read_excel(chemin_fichier_excel)
df.iloc[:, 0] = df.iloc[:, 0].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
colonnes_a_traiter = df.columns[3:33] 

# Calculer la médiane pour chaque colonne spécifiée
medianes = df[colonnes_a_traiter].median()

# Remplacer les cellules vides par les médianes correspondantes
df[colonnes_a_traiter] = df[colonnes_a_traiter].fillna(medianes)
def processv2(df):
    data = df['geometry']
    liste=[]
    for ligne in data:
        sl=[]
        ligne=ligne[10:-2]
        polygone = ligne.split(',')
        for element in polygone:
            a,b= [i for i in element.split(' ') if i != '']
            sl.append((float(a),float(b)))
        liste.append(sl)
    return liste

from shapely.geometry import Polygon
def calculer_aire_polygone(sommets):
    """
    Calcule l'aire d'un polygone à l'aide de la bibliothèque shapely.
    
    :param sommets: Liste des sommets du polygone sous forme de tuples (x, y)
    :return: L'aire du polygone
    """

    polygone = Polygon(sommets)
    aire = polygone.area
    return aire
def perim(sommets):
    polygone = Polygon(sommets)
# Calculer le périmètre du polygone
    perimetre = polygone.length
    return perimetre
def calculate_height_width_ratio(sommets):
    polygon = Polygon(sommets)
    bounds = polygon.bounds
    height = bounds[3] - bounds[1]  # ymax - ymin
    width = bounds[2] - bounds[0]   # xmax - xmin
    return height / width

def calculate_centroid_x(sommets):
    polygon = Polygon(sommets)
    centroid = polygon.centroid
    return centroid.x

def calculate_centroid_y(sommets):
    polygon = Polygon(sommets)
    centroid = polygon.centroid
    return centroid.y
# Calculer l'aire de chaque polygone dans la liste
liste_aires = [calculer_aire_polygone(polygone) for polygone in processv2(df)]
liste_perim = [perim(polygone) for polygone in processv2(df)]
liste_ratio = [calculate_height_width_ratio(polygone) for polygone in processv2(df)]
liste_centroid_x = [calculate_centroid_x(polygone) for polygone in processv2(df)]
liste_centroid_y = [calculate_centroid_y(polygone) for polygone in processv2(df)]
# Ajouter la nouvelle colonne au DataFrame
df['Area'] = liste_aires
df['Perimeter']=liste_perim
df['ratio']=liste_ratio
df['x_centroid']=liste_centroid_x
df['y_centroid']=liste_centroid_y
df = df.drop(['geometry'],axis=1)
df.iloc[:, 0] = df.iloc[:, 0].str.replace('N,A', 'NSP_urban')

df.iloc[:, 1] = df.iloc[:, 1].str.replace('N,A', 'NSP_geo')
one_hot_encoded = df['urban_type'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded)

one_hot_encoded = df['geography_type'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded)


colonnes_a_supprimer = ['urban_type', 'geography_type','index']  # Remplacez par les noms des colonnes à supprimer
df = df.drop(columns=colonnes_a_supprimer)

df['date0'] = pd.to_datetime(df['date0'], format='%d-%m-%Y')
df['date1'] = pd.to_datetime(df['date1'], format='%d-%m-%Y')
df['date2'] = pd.to_datetime(df['date2'], format='%d-%m-%Y')
df['date3'] = pd.to_datetime(df['date3'], format='%d-%m-%Y')
df['date4'] = pd.to_datetime(df['date4'], format='%d-%m-%Y')

# Calculer l'écart en nombre de jours entre les deux dates
df['Ecart_jours1'] = (df['date1'] - df['date0']).dt.days
df['Ecart_jours2'] = (df['date2'] - df['date1']).dt.days
df['Ecart_jours3'] = (df['date3'] - df['date2']).dt.days
df['Ecart_jours4'] = (df['date4'] - df['date3']).dt.days

colonnes_a_supprimer = ['date0','date1','date2','date3','date4']  
df = df.drop(columns=colonnes_a_supprimer)

# Appliquer le One-Hot Encoding sur la colonne 'Categories'
one_hot_encoded0 = df['change_status_date0'].str.get_dummies(sep=',')
one_hot_encoded1 = df['change_status_date1'].str.get_dummies(sep=',')
one_hot_encoded2 = df['change_status_date2'].str.get_dummies(sep=',')
one_hot_encoded3 = df['change_status_date3'].str.get_dummies(sep=',')
one_hot_encoded4 = df['change_status_date4'].str.get_dummies(sep=',')

df = df.join(one_hot_encoded0,rsuffix='0')
df = df.join(one_hot_encoded1,rsuffix='1')
df = df.join(one_hot_encoded2,rsuffix='2')
df = df.join(one_hot_encoded3,rsuffix='3')
df = df.join(one_hot_encoded4,rsuffix='4')

colonnes_a_traiter=['Ecart_jours1','Ecart_jours2','Ecart_jours3','Ecart_jours4']
medianes = df[colonnes_a_traiter].median()

# Remplacer les cellules vides par les médianes correspondantes
df[colonnes_a_traiter] = df[colonnes_a_traiter].fillna(medianes)
colonnes_a_supprimer = ['change_status_date0','change_status_date1','change_status_date2','change_status_date3','change_status_date4'] 
df = df.drop(columns=colonnes_a_supprimer)
couleurs = ['red', 'blue', 'green']

# Créer une liste des dates à considérer (date1, date2, date3, date4, date5)
dates = ['date1', 'date2', 'date3', 'date4', 'date5']

# Parcourir chaque couleur
for couleur in couleurs:
    # Créer une liste pour stocker les moyennes de chaque ligne
    moyennes = []
    # Parcourir chaque ligne
    for index, row in df.iterrows():
        # Créer une liste pour stocker les valeurs de chaque date pour cette couleur
        valeurs_couleur = []
        # Parcourir chaque date
        for date in dates:
            # Récupérer le nom de la colonne
            nom_colonne = f'img_{couleur}_mean_{date}'
            # Ajouter la valeur de la colonne à la liste des valeurs de cette couleur
            valeurs_couleur.append(row[nom_colonne])
        # Calculer la moyenne des valeurs pour cette ligne
        moyenne_ligne = sum(valeurs_couleur) / len(valeurs_couleur)
        # Ajouter la moyenne à la liste des moyennes
        moyennes.append(moyenne_ligne)
    # Créer une nouvelle colonne dans le DataFrame pour stocker les moyennes de cette couleur
    nom_nouvelle_colonne = f'moyenne_{couleur}'
    df[nom_nouvelle_colonne] = moyennes
for couleur in couleurs:
    # Créer une liste pour stocker les moyennes de chaque ligne
    moyennes = []
    # Parcourir chaque ligne
    for index, row in df.iterrows():
        # Créer une liste pour stocker les valeurs de chaque date pour cette couleur
        valeurs_couleur = []
        # Parcourir chaque date
        for date in dates:
            # Récupérer le nom de la colonne
            nom_colonne = f'img_{couleur}_std_{date}'
            # Ajouter la valeur de la colonne à la liste des valeurs de cette couleur
            valeurs_couleur.append(row[nom_colonne])
        # Calculer la moyenne des valeurs pour cette ligne
        moyenne_ligne = sum(valeurs_couleur) / len(valeurs_couleur)
        # Ajouter la moyenne à la liste des moyennes
        moyennes.append(moyenne_ligne)
    # Créer une nouvelle colonne dans le DataFrame pour stocker les moyennes de cette couleur
    nom_nouvelle_colonne = f'std_{couleur}'
    df[nom_nouvelle_colonne] = moyennes
df.to_excel('test_final5.xlsx',index=False)

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Charger toutes vos données à partir du fichier Excel
X = pd.read_excel('train_final5.xlsx')

# Séparation des features et de la variable cible
X_all = X.drop(columns=['change_type'])
y_all = X['change_type']

# Encoder les noms de classe en valeurs numériques
label_encoder = LabelEncoder()
y_all_encoded = label_encoder.fit_transform(y_all)

# Création du dataset XGBoost avec toutes les données
dall = xgb.DMatrix(X_all, label=y_all_encoded)

# Définition des paramètres du modèle
params = {
    'objective': 'multi:softmax',  # Classification multi-classe
    'num_class': len(set(y_all_encoded)),  # Nombre de classes
    'eval_metric': 'mlogloss',  # Métrique d'évaluation pour le F1-score
    'max_depth': 8,
    'learning_rate': 0.24
    
}

# Entraînement du modèle avec 1695 arbres sur toutes les données
num_rounds = 1695
bst = xgb.train(params, dall, num_rounds)

# Charger les nouvelles données pour lesquelles vous souhaitez faire des prédictions
new_data = pd.read_excel('test_final5.xlsx')

# Prétraiter les nouvelles données de la même manière que les anciennes données
X_new = new_data
# Pas besoin de transformer y_new car c'est ce que nous voulons prédire

# Créer un dataset XGBoost pour les nouvelles données
dnew = xgb.DMatrix(X_new)

# Faire des prédictions sur les nouvelles données
y_pred_encoded = bst.predict(dnew)

# Conversion des prédictions encodées en noms d'origine
y_pred = label_encoder.inverse_transform(y_pred_encoded.astype(int))

# Afficher les prédictions
print("Prédictions sur les nouvelles données :")
print(y_pred)

import numpy as np

# Dictionnaire de correspondance
correspondance = {
    'Demolition': 0,
    'Road': 1,
    'Residential': 2,
    'Commercial': 3,
    'Industrial': 4,
    'Mega Projects': 5
}

# Fonction pour mapper les valeurs
def mapper(valeur):
    return correspondance.get(valeur, valeur)  # Retourne la valeur correspondante ou la valeur elle-même si elle n'est pas dans le dictionnaire

# Application de la transformation à l'array
y_pred = np.vectorize(mapper)(y_pred)
## Save results to submission file
pred_df = pd.DataFrame(y_pred)
pred_df.to_csv("sample_submissionFINALRENDU.csv", index=True, index_label='Id')

In [ ]:
#VOTING

from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
import pandas as pd
# Charger les données à partir du fichier Excel
data = pd.read_excel('train_final5.xlsx')

# Séparation des features et de la variable cible
X = data.drop(columns=['change_type'])
y = data['change_type']
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)
# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# Créer différents modèles XGBoost
model1 = XGBClassifier(max_depth=8, n_estimators=1700,learning_rate = 0.25)
model2 = XGBClassifier(max_depth=8, n_estimators=1695,learning_rate = 0.24)
model3 = XGBClassifier(max_depth=8, n_estimators=1705,learning_rate = 0.25)

# Créer un VotingClassifier avec les modèles XGBoost
voting_clf = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')

# Entraîner le VotingClassifier
voting_clf.fit(X, y)

# Charger les nouvelles données pour lesquelles vous souhaitez faire des prédictions
new_data = pd.read_excel('test_final5.xlsx')

# Prétraiter les nouvelles données de la même manière que les anciennes données
X_new = new_data
# Pas besoin de transformer y_new car c'est ce que nous voulons prédire

# Créer un dataset XGBoost pour les nouvelles données
dnew = xgb.DMatrix(X_new)

# Faire des prédictions sur les nouvelles données
y_pred_encoded = voting_clf.predict(X_new)

# Conversion des prédictions encodées en noms d'origine
y_pred = label_encoder.inverse_transform(y_pred_encoded.astype(int))

# Afficher les prédictions
print("Prédictions sur les nouvelles données :")
print(y_pred)
import numpy as np

# Dictionnaire de correspondance
correspondance = {
    'Demolition': 0,
    'Road': 1,
    'Residential': 2,
    'Commercial': 3,
    'Industrial': 4,
    'Mega Projects': 5
}

# Fonction pour mapper les valeurs
def mapper(valeur):
    return correspondance.get(valeur, valeur)  # Retourne la valeur correspondante ou la valeur elle-même si elle n'est pas dans le dictionnaire

# Application de la transformation à l'array
y_pred = np.vectorize(mapper)(y_pred)
## Save results to submission file
pred_df = pd.DataFrame(y_pred)
pred_df.to_csv("sample_submissionRENDUFINALVOTING.csv", index=True, index_label='Id')



In [ ]:
#EXEMPLE DE GRIDSEARCH


import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Charger les données à partir du fichier Excel
data = pd.read_excel('train_final5.xlsx')

# Séparation des features et de la variable cible
X = data.drop(columns=['change_type'])
y = data['change_type']
# Encoder les noms de classe en valeurs numériques
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Définition des paramètres à tester
param_grid = {
    'learning_rate': [ 0.25,0.24,0.26],
    'max_depth': [ 8],
    'n_estimators': [1700,1695],
    'gamma': [ 0],
    'alpha': [0],
    'lambda': [ 0]
}
   

# Création du classifieur XGBoost
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y)) ,seed=42)

# Recherche par grille pour trouver les meilleurs paramètres
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, cv=3, scoring='f1_macro')
grid_search.fit(X_train, y_train)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres trouvés:")
print(grid_search.best_params_)

# Prédictions sur l'ensemble de test avec les meilleurs paramètres
best_xgb_classifier = grid_search.best_estimator_
y_pred = best_xgb_classifier.predict(X_test)

# Calcul de la précision
accuracy = f1_score(y_test, y_pred,average='micro')
print("Précision du modèle avec les meilleurs paramètres :", accuracy)

# Affichage du rapport de classification
print("Rapport de classification :")
print(classification_report(y_test, y_pred))